In [8]:
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import tensorflow_hub as hub
import functools

print(tf.__version__)
print(tfds.__version__)
print(hub.__version__)

2.6.2
4.5.2
0.12.0


In [9]:
ds = tfds.load("imagenette")
def extract_image(example):
    image = example['image']
    return image

def preprocess_image(image, height, width):
    image = tf.image.resize_with_crop_or_pad(image, target_height=height, target_width=width)
    image = tf.cast(image, tf.float32) / 255.0
    return image


def get_image_batches(batch_size=128, height=256, width=256):
    partial_preprocess_image = functools.partial(preprocess_image, height=height, width=width)
    train_ds = ds['train']
    train_ds = ( train_ds.map(extract_image)
                .map(partial_preprocess_image)
                .cache()
                .shuffle(buffer_size=1000)
                .batch(batch_size)
                .prefetch(tf.data.AUTOTUNE)
                )
    return train_ds

2022-03-30 00:41:16.643004: W tensorflow/core/platform/cloud/google_auth_provider.cc:184] All attempts to get a Google authentication bearer token failed, returning an empty token. Retrieving token from files failed with "Not found: Could not locate the credentials file.". Retrieving token from GCE failed with "Failed precondition: Error executing an HTTP request: libcurl code 6 meaning 'Couldn't resolve host name', error details: Could not resolve host: metadata".


Dl Completed...: 0 url [00:00, ? url/s]

Dl Size...: 0 MiB [00:00, ? MiB/s]

Extraction completed...: 0 file [00:00, ? file/s]

Generating splits...:   0%|          | 0/2 [00:00<?, ? splits/s]

...

Shuffling /home/jangedoo/tensorflow_datasets/imagenette/full-size-v2/1.0.0.incompleteDEV618/imagenette-train.t…

Generating validation examples...:   0%|          | 0/3925 [00:00<?, ? examples/s]

Shuffling /home/jangedoo/tensorflow_datasets/imagenette/full-size-v2/1.0.0.incompleteDEV618/imagenette-validat…

Dataset imagenette downloaded and prepared to /home/jangedoo/tensorflow_datasets/imagenette/full-size-v2/1.0.0. Subsequent calls will reuse this data.


2022-03-30 00:44:58.509101: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-03-30 00:44:58.509581: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-03-30 00:44:58.510105: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-03-30 00:44:58.510753: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:923] could not open file to read NUMA node: /sys/bus/pci/devices/0000:26:00.0/numa_node
Your kernel may have been built without NUMA support.
2022-03-30 00:44:58.510767: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1594] Could not ident

In [10]:
BATCH_SIZE = 32
IMG_WIDTH = IMG_HEIGHT = 256
train_ds = get_image_batches(batch_size=BATCH_SIZE, height=IMG_HEIGHT, width=IMG_WIDTH)

In [11]:
images = np.array([img for batch in train_ds.take(20) for img in batch]) # take 20 batches 
print(images.shape)

2022-03-30 00:44:59.336200: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:185] None of the MLIR Optimization Passes are enabled (registered 2)
2022-03-30 00:44:59.805480: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 23970816 exceeds 10% of free system memory.
2022-03-30 00:44:59.808980: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 31166226 exceeds 10% of free system memory.
2022-03-30 00:45:00.537543: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 25165824 exceeds 10% of free system memory.
2022-03-30 00:45:00.570503: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 25165824 exceeds 10% of free system memory.
2022-03-30 00:45:00.601760: W tensorflow/core/framework/cpu_allocator_impl.cc:80] Allocation of 25165824 exceeds 10% of free system memory.
2022-03-30 00:45:01.294674: W tensorflow/core/kernels/data/cache_dataset_ops.cc:768] The calling iterator did not fully read the dataset being c

(640, 256, 256, 3)


# Vectorization

- load model
- extract features using model

In [12]:
vectorizer = tf.keras.Sequential([
    hub.KerasLayer("https://tfhub.dev/google/imagenet/inception_resnet_v2/feature_vector/5", trainable=False)
])
vectorizer.build([None, IMG_HEIGHT, IMG_WIDTH, 3])

In [13]:
features = vectorizer.predict(images, batch_size=BATCH_SIZE)
print(features.shape)

2022-03-30 00:45:32.263662: I tensorflow/stream_executor/cuda/cuda_dnn.cc:369] Loaded cuDNN version 8201
2022-03-30 00:45:32.770635: I tensorflow/core/platform/default/subprocess.cc:304] Start cannot spawn child process: No such file or directory


(640, 1536)


# Image search using KNN

In [15]:
from sklearn.neighbors import NearestNeighbors
knn = NearestNeighbors(n_neighbors=5, metric="cosine")
knn.fit(features)

NearestNeighbors(metric='cosine')

In [18]:
def show_similar_images(image_idx, n_neighbors=10):
    input_images = images[image_idx:image_idx+5]
    features = vectorizer.predict(input_images)
    knn_output = knn.kneighbors(features, n_neighbors=n_neighbors)
    
    images_with_distances_and_nbors = zip(input_images, *knn_output)
    
    fig, axes = plt.subplots(len(input_images), n_neighbors+1, figsize=(20, len(input_images)*3))
    
    for i, (image, distances, nbors) in enumerate(images_with_distances_and_nbors):
        for j in range(n_neighbors+1):
            ax = axes[i, j]
            img = image if j==0 else images[nbors[j-1]]
            if j == 0:
                ax.set_title("Input Image")
            else:
                ax.set_title(f"Sim: {1-distances[j-1]:.2f}")
            ax.set_axis_off()
            ax.imshow(img)

import ipywidgets as w
_ = w.interact(show_similar_images, 
    image_idx=w.IntSlider(max=len(images)-1, continuous_update=False),
    n_neighbors=w.IntSlider(min=2, value=5, max=10, continuous_update=False),
)

interactive(children=(IntSlider(value=0, continuous_update=False, description='image_idx', max=639), IntSlider…